In [ ]:
import pandas as pd
import numpy as np
import ast
import iqplot
import bokeh.layouts
import bokeh.models
import bokeh.plotting
import holoviews as hv
import panel as pn
import colorcet

from holoviews.operation.timeseries import rolling

hv.extension('bokeh')
notebook_url='localhost:8889'
bokeh.io.output_notebook()

In [ ]:
Data = pd.read_csv(
    "/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiSensory_Project/GatedArenas/Results/DataSetNew.csv"
)



ThreshSlider = pn.widgets.IntSlider(name='ThreshSlider', value=80, start=60, end=270, step=10)

ThreshSlider

In [ ]:
Data.head()

In [ ]:
def slider_callback(ThreshSlider):
    

    for index, row in Data.iterrows():
        # print(row['Durations Left Corner'])

        # print (1 for i in row['Durations Left Corner'])
        Data.loc[index, "Peeks Left"] = sum(
            1 for i in ast.literal_eval(row["Durations Left Corner"]) if i > ThreshSlider
        )
        # Data_noWater_Simple['Peeks Left'][rows]= sum(1 for i in Data['Durations Left Corner'][rows] if i > param)
    #print(Data['Peeks Left'])
    box = hv.BoxWhisker(data=Data,
                    kdims="Training",
                    vdims="Peeks Left").opts(framewise=True,
                                            ylim=(0, 40),
                                            box_fill_alpha=0,
                                            invert_axes=True,
                                            invert_yaxis=True,
                                            #box_line_color="gray",
                                            )
    points = hv.Scatter(data=Data,
                    kdims="Training",
                    vdims="Peeks Left").opts(framewise=True,
                                             cmap=colorcet.b_glasbey_category10,
                                             invert_axes=True,
                                            invert_yaxis=True,
                                            ylim=(0, 40),
                                            color="Training",
                                            jitter=0.4,
                                            )
    
    
    return box * points

dmap = hv.DynamicMap(pn.bind(slider_callback, ThreshSlider=ThreshSlider))
#dmap = hv.DynamicMap(slider_callback)

app = pn.Row(pn.WidgetBox('## Threshold Explorer', ThreshSlider), 
             dmap.opts(width=500, 
                       framewise=True,
                      )).servable()
app

In [ ]:
iqplot.stripbox(data=df, q="mpg", cats="origin", title="strip-box")

In [ ]:
def slider_callback(ThreshSlider):
    

    for index, row in Data.iterrows():
        # print(row['Durations Left Corner'])

        # print (1 for i in row['Durations Left Corner'])
        Data.loc[index, "Peeks Left"] = sum(
            1 for i in ast.literal_eval(row["Durations Left Corner"]) if i > ThreshSlider
        )
        # Data_noWater_Simple['Peeks Left'][rows]= sum(1 for i in Data['Durations Left Corner'][rows] if i > param)
    #print(Data['Peeks Left'])
    
    return iqplot.stripbox(data=Data,
                           q='Peeks Left',
                           cats='Training').opts(framewise=True)

dmap = hv.DynamicMap(pn.bind(slider_callback, ThreshSlider=ThreshSlider))
#dmap = hv.DynamicMap(slider_callback)

app = pn.Row(pn.WidgetBox('## Threshold Explorer', ThreshSlider), 
             dmap.opts(width=500, 
                       framewise=True,
                      )).servable()
app

In [ ]:
# Prep dataset from data
Melted = pd.melt(
    Data,
    id_vars=["Training", "ObjectsReinforced", "Reinforced_side", "Date", "Fly"],
    value_name="Durations",
    value_vars=[

        "Durations Left Corner",

        "Durations Right Corner",

        "Durations Top Corner",

        "Durations Left Front",

        "Durations Right Front",

        "Durations Top Front",
    ],
    var_name="Variable",
)


conditions = [
    (
        Melted["Reinforced_side"].str.contains("Right")
        & (Melted["Variable"].str.contains("Right")
    )
    ),
    (
        Melted["Reinforced_side"].str.contains("Left")
        & (Melted["Variable"].str.contains("Left")
    )
    ),
    (
        Melted["Reinforced_side"].str.contains("Right")
        & (Melted["Variable"].str.contains("Left")
    )
    ),
    (
        Melted["Reinforced_side"].str.contains("Left")
        & (Melted["Variable"].str.contains("Right")
     )
    ),
    (Melted["Variable"].str.contains("Top")
     ),

]

values = [
    "Rewarded Side",
    "Rewarded Side",
    "Punished Side",
    "Punished Side",
    "Empty Side",
]
Melted["Condition"] = np.select(conditions, values)

conditions = [
    (
            (Melted["Variable"].str.contains("Corner"))
    ),
    (
            (Melted["Variable"].str.contains("Front")))

]
values = [
    "Corner",
    "Front",
]
Melted["Location"] = np.select(conditions, values)

ThreshSlider = pn.widgets.IntSlider(name='ThreshSlider', value=80, start=60, end=270, step=10)

Condis = list(Melted['Condition'].unique())
Condis.append('All')

Condition = pn.widgets.RadioButtonGroup(options=Condis)

Locs = list(Melted['Location'].unique())
Locs.append('All')

Location = pn.widgets.Select(options=Locs) 

Dates = list(Melted["Date"].unique())
Dates.insert(0, "All")

Date = pn.widgets.Select(options=Dates)

def slider_callback(Condition, Location, ThreshSlider):
    
    if ((Condition == "All") & (Location == "All")):
        Subset = Melted
        
    elif ((Condition == "All") & (Location != "All")):
        Subset = Melted[(Melted["Location"] == Location)]
            
    elif ((Condition != "All") & (Location == "All")):
        Subset = Melted[(Melted["Condition"] == Condition)]
    elif ((Condition != "All") & (Location != "All")): 
        Subset = Melted[(Melted["Condition"] == Condition) & (Melted["Location"] == Location)]
    
    
    for index, row in Subset.iterrows():
        # print(row['Durations Left Corner'])

        # print (1 for i in row['Durations Left Corner'])
        Subset.loc[index, "Peeks"] = sum(
            1 for i in ast.literal_eval(row["Durations"]) if i > ThreshSlider
        )
        # Data_noWater_Simple['Peeks Left'][rows]= sum(1 for i in Data['Durations Left Corner'][rows] if i > param)
    # print(Data['Peeks Left'])
    box = hv.BoxWhisker(data=Subset,
                        kdims=['Training'],
                        vdims=["Peeks"]).opts(framewise=True,
                                                 ylim=(0, 40),
                                                 box_fill_alpha=0,
                                                 invert_axes=True,
                                                 invert_yaxis=True,

                                                 # box_line_color="gray",
                                                 )
    points = hv.Scatter(data=Subset,
                        kdims=['Training'],
                        vdims=["Peeks"]).opts(framewise=True,
                                                 cmap=colorcet.b_glasbey_category10,
                                                 invert_axes=True,
                                                 invert_yaxis=True,
                                                 ylim=(0, 40),
                                                 color="Training",
                                                 jitter=0.4,
                                                 size=6,
                                                 alpha=0.5,
                                                 tools=['hover'],
                                                 )

    return box * points


dmap = hv.DynamicMap(pn.bind(slider_callback,Condition=Condition,Location=Location , ThreshSlider=ThreshSlider))
# dmap = hv.DynamicMap(slider_callback)

app = pn.Row(pn.WidgetBox('## Threshold Explorer', ThreshSlider, Condition, Location),
             pn.Spacer(width=50),
             dmap.opts(width=500,
                       framewise=True,
                       )).servable()

app

#pn.serve(app)


In [ ]:
Condis = list(Melted["Condition"].unique())
Condis.append("All")

Condition = pn.widgets.RadioButtonGroup(options=Condis,
                                        #margin=25,
                                        width=150,
                                        )

Condition

In [ ]:
Melted.head()

In [ ]:
list(Condis)

In [ ]:
Locs.append("All")
Locs